**LLM Strengths:**

Reasoning Capability: LLMs can understand and break down questions to determine how to respond (ability to think).

Language Generation: LLMs generate word-by-word responses based on reasoning (ability to speak).

**LLM Limitations:**

LLMs cannot perform tasks beyond thinking and generating content, as they lack the ability to execute actions (no hands or legs analogy).

Examples of tasks LLMs cannot do:

Fetch live weather data.

Reliably solve complex math problems (basic arithmetic is possible but not guaranteed for complex tasks).

Call external APIs (e.g., post a tweet on Twitter).

Run code or interact with databases.

*Analogy*: An LLM is like a human body with a brain (to think and speak) but no hands or legs (to act).

# TOOLS



**Definition of Tools:**

Tools are mechanisms that give LLMs the ability to execute tasks (providing hands and legs).

Technically, tools are Python functions with task-specific logic, packaged to interact with LLMs.

Example: A tool for booking train tickets on IRCTC’s website can be created and connected to an LLM. When asked to book a ticket, the LLM uses the tool to perform the task.

Power of Tools: The more tools you add to an LLM, the more types of tasks it can perform.

**Types of Tools in LangChain**

**Built-in Tools:**
Pre-built tools provided by the LangChain team for common use cases, requiring no coding.

Examples:
DuckDuckGo Search: Perform web searches using the DuckDuckGo search engine.

Wikipedia Query Run: Search Wikipedia and retrieve summarized content.

Python REPL Tool: Run raw Python code (e.g., calculate factorial reliably).

Gmail Send Message Tool: Send emails via Gmail.


Advantages: Production-ready, minimal setup, no need to write function logic.

Use Case: Useful for common tasks like web searching, code execution, or database querying.

https://python.langchain.com/docs/integrations/tools/

In [2]:
from langchain_community.tools import DuckDuckGoSearchRun

In [4]:
search = DuckDuckGoSearchRun()
search.invoke("Latest news in india today?")

'Stay here for real-time updates on breaking news from India and across the world that you can\'t miss: India\'s forex reserves rise by $ 1.983 billion to $ 688.129 billion during the week ended ... "Officers and officials of CGI Shanghai, friends of India and members of Indian diaspora led by Consul General @PratikMathur1 offered deepest and heartfelt condolences to the victims of the #PahalgamTerroristAttack today in the Consulate," stated a post by the Consulate on social media platform X, dated April 30. India News | Latest India News | Read latest and breaking news from India. Today\'s top India news headlines, news on Indian politics, elections, government, business, technology, and Bollywood. Pakistan might issue a formal diplomatic notice to India in the wake of the suspension of Indus Water Treaty, news agency PTI reported citing a media report on Friday. The decision to issue the notice was taken after discussions between Pakistan\'s ministries of Foreign Affairs, Law, and Wa

In [6]:
print(search.name)
print(search.description)
print(search.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


**Custom Tools:**
Tools created by users for specific use cases not covered by built-in tools.

When to Create Custom Tools:

Calling Proprietary APIs: Integrating with a company’s internal APIs (e.g., a travel booking API for a MakeMyTrip-like application).

Encapsulating Business Logic: Implementing unique logic specific to an application.

Interacting with Databases/Products: Enabling an LLM to interact with a custom database or app.

Example: Building a tool for an agent to perform bookings by interacting with a company’s database via APIs.



**Methods to Create Custom Tools**:


**Using the @tool Decorator (Simplest Method)**:
Process (Three Steps):
Write a Function: Create a Python function with the task logic (e.g., a function to multiply two numbers).
Add a docstring (highly recommended) to describe what the function does, as it helps the LLM understand the tool’s purpose.

Add Type Hints: Specify input and output types (e.g., int for inputs a and b, and int for the return value).
Type hints are optional but recommended to clarify data expectations for the LLM.

Apply @tool Decorator: Add the @tool decorator above the function to make it a special function that can interact with an LLM.

In [8]:
from langchain.tools import tool

@tool
def multiply(a: int,b: int)->int:
    '''Multiplication of two numbers'''
    return a*b

multiply.invoke({'a':3,'b':4})

12

In [9]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiplication of two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [10]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiplication of two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


**Using the StructuredTool Class with Pydantic (More Strict Method)**:
Definition: A structured tool enforces a strict input schema using a Pydantic model, making it more robust for production.

Advantages:
Enforces stricter input constraints using Pydantic, making it suitable for production-ready applications.

Separates function logic from schema definition for better organization.

Use Case: Preferred for complex applications requiring robust input validation.

In [23]:
from pydantic import BaseModel,Field
class Multiply(BaseModel):
    a : int = Field(description='First Number',required=True)
    b : int = Field(description="Second Number",required=True)

In [24]:
def multiply_func(a: int, b: int) -> int:
    return a * b

In [25]:
from langchain.tools import StructuredTool
multiply_tool = StructuredTool.from_function(
    func = multiply_func,
    name = "multiply",
    description = "Multiplication of two numbers",
    args_schema = Multiply
    
)

In [26]:
multiply_tool.invoke({'a':3, 'b':3})

9

**Using the BaseTool Class (Most Customizable Method)**:
Definition: The BaseTool class is the abstract base class for all LangChain tools, defining the core structure and interface. All tools (built-in or custom) inherit from it.

Advantages:
Offers deep customization, including support for asynchronous tools.

Ideal for production-level applications with complex requirements or concurrency needs.

Drawback: More complex than the @tool or StructuredTool methods.

Use Case: Used when advanced customization or asynchronous functionality is required.

In [27]:
from langchain.tools import BaseTool
from typing import Type

In [17]:
# arg schema using pydantic

class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [28]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

In [29]:
MultiplyTool().invoke({'a':6, 'b':3})

18

**Recommendation:**
For most scenarios (80-90%), the @tool decorator is sufficient and simplest.

For production-ready applications, consider StructuredTool or BaseTool for stricter constraints or advanced features.

Explore LangChain’s documentation for additional methods and examples.

### TOOLKIT

**Toolkits**
Definition: A toolkit is a collection of related tools that serve a common purpose, packaged together for convenience and reusability.

Purpose:
Organize multiple tools that are logically related.

Enhance reusability across different applications.

Example:
Google Drive Toolkit: Combines tools like:
Upload files to Google Drive.

Search files on Google Drive.

Read files from Google Drive.

These tools are related as they all interact with Google Drive.



**Built-in Toolkits**: LangChain provides prebuilt toolkits for common use cases.

Benefits:
Simplifies access to multiple related tools.

Promotes reusability across applications.

Note: Toolkits are a small but useful concept for organizing tools efficiently.



In [30]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [31]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]

In [22]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)


add => Add two numbers
multiply => Multiply two numbers


**Why Tools Are Critical for Agents:**

The combination of LLMs (for reasoning) and tools (for action) creates an agent.

Tools are as essential as LLMs for building agents, making the concept of tools foundational.



# TOOL CALLING


**Definition:** Tool calling is the process where an LLM decides during a conversation or task that a specific tool is needed and generates a structured output containing the tool’s name and required input arguments.

**Purpose:** Enables LLMs to suggest which tool to use and how to use it, allowing task execution without the LLM directly performing the action.

**Key Points:**
LLMs do not execute tools; they only recommend the tool and provide input arguments.

Actual tool execution is handled by LangChain and the programmer, ensuring control and safety.

Example: For a query like “What’s 8 * 7?”, the LLM identifies a multiplication tool, specifies inputs (e.g., a=8, b=7), and suggests invoking it.

### Tool Binding


**Definition:** Tool binding is the process of registering tools with an LLM, so the LLM knows which tools are available, what they do, and how to invoke them.

**Purpose:**
Informs the LLM about available tools.

Provides tool descriptions to clarify their functionality.

Specifies the input schema (format) each tool expects.

**Outcome:** After binding, the LLM can intelligently select and suggest tools during conversations based on their descriptions and input requirements.

**Limitation:** Not all LLMs support tool binding; only specific models have this capability.

In [35]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F84F065F40>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F84F067110>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [53]:
@tool
def mul(a:int,b:int) -> int:
    "Multiplication of two integer numbers"
    return a*b 

In [36]:
multiply

StructuredTool(name='multiply', description='Multiply two numbers', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x000001F83809BB00>)

In [37]:
multiply.args_schema.model_json_schema()

{'description': 'Multiply two numbers',
 'properties': {'a': {'title': 'A', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'}},
 'required': ['a', 'b'],
 'title': 'multiply',
 'type': 'object'}

In [54]:
llm_with_tools = llm.bind_tools([mul])

In [55]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F84F065F40>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F84F067110>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [56]:
llm_with_tools

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F84F065F40>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F84F067110>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'mul', 'description': 'Multiplication of two integer numbers', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}]}, config={}, config_factories=[])

### Tool Calling



**Tool Calling Process**
Scenario:
An LLM with access to a tool (e.g., multiplication tool) is queried.

For general queries (e.g., “How are you?”), the LLM responds directly without tool involvement.

For task-specific queries (e.g., “Multiply 3 by 10”), the LLM:
Checks available tools.

Identifies a relevant tool (e.g., multiplication tool).

Generates a structured output with:
Tool name (e.g., multiply).

Input arguments (e.g., {"a": 3, "b": 10}).

Output: The LLM produces a tool call (not a text response), which includes the tool name and arguments, avoiding direct execution to maintain safety.

**Key Insight:** Tool calling is an advisory process; the LLM suggests the tool and inputs but does not execute it, leaving execution to the programmer.

In [57]:
res = llm_with_tools.invoke("Hi, How are you?")
print(res) # Here LLM produced content on its own capabilities, No tools are involved here

content="I'm doing well, thanks for asking!" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 1866, 'total_tokens': 1876, 'completion_time': 0.008333333, 'prompt_time': 0.233369311, 'queue_time': -0.28559678099999997, 'total_time': 0.241702644}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None} id='run--19069624-1461-4c92-859a-03943f988d4a-0' usage_metadata={'input_tokens': 1866, 'output_tokens': 10, 'total_tokens': 1876}


In [ ]:
res = llm_with_tools.invoke("What is multiplication of three and five?")
print(res) # Here LLM produced empty content, instead suggested to call multiply tool (see additional_kwargs,tool_calls in response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_rrtk', 'function': {'arguments': '{"a":3,"b":5}', 'name': 'mul'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 925, 'total_tokens': 997, 'completion_time': 0.06, 'prompt_time': 0.115587023, 'queue_time': 0.08821108000000001, 'total_time': 0.175587023}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--dc370bd0-064a-4e0f-aacf-db11993646d3-0' tool_calls=[{'name': 'mul', 'args': {'a': 3, 'b': 5}, 'id': 'call_rrtk', 'type': 'tool_call'}] usage_metadata={'input_tokens': 925, 'output_tokens': 72, 'total_tokens': 997}


In [59]:
res.tool_calls

[{'name': 'mul',
  'args': {'a': 3, 'b': 5},
  'id': 'call_rrtk',
  'type': 'tool_call'}]

### Tool Calling

**Definition:** Tool execution is the step where the programmer invokes the suggested tool using the input arguments provided by the LLM during tool calling.

**Process:**
The LLM’s tool call output (e.g., tool name and arguments) is used to invoke the tool.

The tool runs its logic and returns a result.

The result is packaged as a tool message, a special message type containing the tool’s output.

**Role in Workflow:**
Completes the task execution initiated by the LLM’s suggestion.

The tool message can be sent back to the LLM to provide context for generating a final response.

In [61]:
output = multiply.invoke(res.tool_calls[0])
output # Here output is ToolMessage unlike AIMessage

ToolMessage(content='15', name='multiply', tool_call_id='call_rrtk')


**Conversation History and Context**

Concept: A conversation history is maintained to track the sequence of interactions, including:
Human Message: User’s query (e.g., “Multiply 3 by 10”).

AI Message: LLM’s response, which may include tool calls.

Tool Message: Result of tool execution.

Purpose: The history provides full context to the LLM, enabling it to generate a final answer by combining user queries, tool suggestions, and execution results.

**Workflow:**

User sends a query (human message).

LLM responds with a tool call (AI message).

Programmer executes the tool, generating a tool message.

All messages are sent back to the LLM, which generates a final response (e.g., “The product of 3 and 10 is 30”).

In [64]:
from langchain_core.messages import HumanMessage

In [78]:
question = "what is multiplication of three and four?"
message = HumanMessage(question)
history = []
history.append(message)

In [79]:
response1 = llm_with_tools.invoke(history)

In [80]:
history.append(response1)

In [81]:
history

[HumanMessage(content='what is multiplication of three and four?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_czpe', 'function': {'arguments': '{"a":3,"b":4}', 'name': 'mul'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 925, 'total_tokens': 1004, 'completion_time': 0.065833333, 'prompt_time': 0.135056495, 'queue_time': 0.088369968, 'total_time': 0.200889828}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7c96bfc9-2445-4dac-a528-69e439d3fb10-0', tool_calls=[{'name': 'mul', 'args': {'a': 3, 'b': 4}, 'id': 'call_czpe', 'type': 'tool_call'}], usage_metadata={'input_tokens': 925, 'output_tokens': 79, 'total_tokens': 1004})]

In [82]:
response2 = mul.invoke(response1.tool_calls[0])
response2

ToolMessage(content='12', name='mul', tool_call_id='call_czpe')

In [83]:
history

[HumanMessage(content='what is multiplication of three and four?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_czpe', 'function': {'arguments': '{"a":3,"b":4}', 'name': 'mul'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 925, 'total_tokens': 1004, 'completion_time': 0.065833333, 'prompt_time': 0.135056495, 'queue_time': 0.088369968, 'total_time': 0.200889828}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7c96bfc9-2445-4dac-a528-69e439d3fb10-0', tool_calls=[{'name': 'mul', 'args': {'a': 3, 'b': 4}, 'id': 'call_czpe', 'type': 'tool_call'}], usage_metadata={'input_tokens': 925, 'output_tokens': 79, 'total_tokens': 1004})]

In [ ]:
history.append(response2)

In [87]:
history

[HumanMessage(content='what is multiplication of three and four?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_czpe', 'function': {'arguments': '{"a":3,"b":4}', 'name': 'mul'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 925, 'total_tokens': 1004, 'completion_time': 0.065833333, 'prompt_time': 0.135056495, 'queue_time': 0.088369968, 'total_time': 0.200889828}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7c96bfc9-2445-4dac-a528-69e439d3fb10-0', tool_calls=[{'name': 'mul', 'args': {'a': 3, 'b': 4}, 'id': 'call_czpe', 'type': 'tool_call'}], usage_metadata={'input_tokens': 925, 'output_tokens': 79, 'total_tokens': 1004}),
 ToolMessage(content='12', name='mul', tool_call_id='call_czpe')]

In [86]:
llm_with_tools.invoke(history)

AIMessage(content='The result is 12.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 992, 'total_tokens': 999, 'completion_time': 0.005833333, 'prompt_time': 0.124763105, 'queue_time': 0.068485642, 'total_time': 0.130596438}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'stop', 'logprobs': None}, id='run--e10bc70b-37d1-4f9e-b879-490603a82e2e-0', usage_metadata={'input_tokens': 992, 'output_tokens': 7, 'total_tokens': 999})


**Injected Tool Arguments**
**Definition:** A mechanism to mark specific tool inputs as “injected,” meaning the LLM does not provide their values during tool calling, and the programmer supplies them later.

**Purpose:**
Ensures sequential tool execution by preventing the LLM from prematurely setting inputs (e.g., conversion rate) based on outdated knowledge.

Allows the programmer to use outputs from one tool (e.g., conversion rate) as inputs for another.

Example:
In the convert tool, the conversion rate is marked as an injected tool argument.

During tool calling, the LLM specifies only the base currency value (e.g., 10 USD) and leaves the conversion rate blank.

The programmer fetches the conversion rate from the get_conversion_factor tool and injects it into the convert tool’s arguments before execution.

Benefit: Maintains logical dependency between tools, ensuring accurate results (e.g., real-time conversion rate is used).

Is This an AI Agent?
Question: Does the currency conversion application qualify as an AI agent?

Answer: No, it is not a true AI agent.

Reason:
AI Agent Characteristics: An AI agent is autonomous, capable of breaking down problems, deciding steps, and executing tasks independently without manual intervention.

Application Limitations: The currency conversion application relies on the programmer to:
Manually execute tools.

Manage conversation history.

Inject tool arguments.

This manual involvement contrasts with an agent’s ability to autonomously handle the entire workflow.

Agent Example:
For “Convert 10 USD to INR,” an AI agent would:
Recognize the need for a conversion rate and call get_conversion_factor.

Use the fetched rate to call convert with appropriate inputs.

Deliver the final result without programmer intervention.

The agent would handle all steps autonomously, unlike the application, which requires coded logic for each step.


**Agent Definition:** An AI agent is an LLM-powered system that can autonomously think, decide, and take actions using external tools and APIs to achieve a goal.

Agent Capabilities:
Reasoning and Decision-Making: Provided by the LLM (thinking step-by-step to solve a problem).

Action Execution: Enabled by tools (performing tasks based on reasoning).